In [26]:
import requests
import os
import json
from dotenv import load_dotenv
from datetime import datetime, timedelta

load_dotenv()

# Configuración API
TOKEN = "Xegy8dVsa1H8SFfojJcwYtDL"

print(f"TOKEN: {os.getenv('BUK_AUTH_TOKEN')}")



TOKEN: nbeU1PWnHXqXJLFGzoh5yiyw


In [27]:
def obtener_empleado(ruts=[]):
    """
    Obtiene uno o varios empleados desde la API usando su RUT.
    Retorna una lista de empleados filtrados.
    """
    empleados_filtrados = []

    for r in ruts:
        URL = f"https://cramer.buk.cl/api/v1/chile/employees?document_number={r}"
        headers = {"auth_token": TOKEN}
        respuesta = requests.get(URL, headers=headers)

        if respuesta.status_code == 200:
            datos = respuesta.json()
            if "data" in datos and len(datos["data"]) > 0:
                empleado_completo = datos["data"][0]

                empleado_filtrado = {
                    "person_id": empleado_completo.get("person_id"),
                    "id": empleado_completo.get("id"),
                    "full_name": empleado_completo.get("full_name"),
                    "rut": empleado_completo.get("rut"),
                    "email": empleado_completo.get("email"),
                    "personal_email": empleado_completo.get("personal_email"),
                    "active_since": empleado_completo.get("active_since"),
                    "status": empleado_completo.get("status"),
                    "payment_method": empleado_completo.get("payment_method"),
                    "id_boss": empleado_completo.get("current_job", {}).get("boss", {}).get("id"),
                    "rut_boss": empleado_completo.get("current_job", {}).get("boss", {}).get("rut"),
                    "contract_type": empleado_completo.get("current_job", {}).get("contract_type"),
                    "start_date": empleado_completo.get("current_job", {}).get("start_date"),
                    "end_date": empleado_completo.get("current_job", {}).get("end_date"),
                    "contract_finishing_date_1": empleado_completo.get("current_job", {}).get("contract_finishing_date_1"),
                    "contract_finishing_date_2": empleado_completo.get("current_job", {}).get("contract_finishing_date_2"),
                }

                empleados_filtrados.append(empleado_filtrado)
            else:
                print(f"⚠️ No se encontró empleado para RUT {r}")
        else:
            print(f"❌ Error {respuesta.status_code} consultando {r}")
            print(respuesta.text)

    return empleados_filtrados



In [24]:
def calcular_fecha_alerta(empleado):
    """
    Calcula la fecha de alerta según el tipo de contrato y status
    """
    tipo_contrato = empleado.get("contract_type", "").lower()
    status = empleado.get("status", "").lower()
    metodo_pago = empleado.get("payment_method", "").lower()
    fecha_activacion = empleado.get("active_since")
    termino_primer_plazo = empleado.get("contract_finishing_date_1")
    termino_segundo_plazo = empleado.get("contract_finishing_date_2")
    
    # Solo considerar empleados activos con transferencia bancaria
    if status != "activo" or metodo_pago != "transferencia bancaria" or tipo_contrato == "indefinido":
        return None
    
    try:
        if fecha_activacion:
            fecha_activacion = datetime.strptime(fecha_activacion, "%Y-%m-%d")
            primer_plazo = datetime.strptime(termino_primer_plazo, "%Y-%m-%d") if termino_primer_plazo else None
            segundo_plazo = datetime.strptime(termino_segundo_plazo, "%Y-%m-%d") if termino_segundo_plazo else None
        else:
            return None
            
        fecha_alerta = None
        motivo = None

        # primera alerta → paso a segundo plazo
        if tipo_contrato == "fijo" and primer_plazo is not None and segundo_plazo is not None:
            fecha_alerta = fecha_activacion + timedelta(days=14)
            motivo = "Paso a segundo plazo de contratación"
        # segunda alerta → paso a indefinido
        elif tipo_contrato == "fijo" and primer_plazo is not None and segundo_plazo is None:
            fecha_alerta = fecha_activacion + timedelta(days=76)
            motivo = "Paso a contrato indefinido"
        
        if fecha_alerta:
            return {
                "fecha_alerta": fecha_alerta.strftime("%Y-%m-%d"),
                "motivo": motivo,
                "dias_desde_inicio": (datetime.now() - fecha_activacion).days,
                "requiere_accion": fecha_alerta <= datetime.now()
            }
        else:
            return None
        
    except ValueError as e:
        print(f"Error procesando fechas para {empleado.get('full_name')}: {e}")
        return None

In [33]:
def procesar_alertas_renovacion(ruts):
    """
    Procesa empleados y genera alertas de renovación
    """
    empleados = obtener_empleado(ruts)  # <- trae la data de la API/BD
    alertas = []
    
    for empleado in empleados:
        alerta = calcular_fecha_alerta(empleado)  
        
        if alerta:  # solo si hay alerta
            empleado_con_alerta = {
                **empleado,   # Todos los datos originales del empleado
                **alerta      # Los datos de la alerta calculada
            }
            alertas.append(empleado_con_alerta)
    
    return alertas


ruts_a_procesar = [194204698,205738428,200060296,256072920,144850440] 
alertas = procesar_alertas_renovacion(ruts_a_procesar)

# Mostrar resultados
for alerta in alertas:
    print(f"\n👤 {alerta.get('full_name', 'Desconocido')}")
    print (f"Fecha de inicio: {alerta.get('active_since', 'N/A')}")
    print(f"Fecha alerta: {alerta['fecha_alerta']}")
    print(f"Motivo: {alerta['motivo']}")
    #print(f"⚠️ Requiere acción: {'SÍ' if alerta['requiere_accion'] else 'NO'}")
    print(f"Jefe (ID): {alerta.get('id_boss')} - RUT: {alerta.get('rut_boss')}")
    print(f"Días en Cramer: {alerta['dias_desde_inicio']} días")



👤 Benjamín Sebastián Gacitúa Godoy
Fecha de inicio: 2025-07-14
Fecha alerta: 2025-09-28
Motivo: Paso a contrato indefinido
Jefe (ID): 4251 - RUT: 12.862.673-5
Días en Cramer: 45 días

👤 Marcelo Del Carmen González Baeza
Fecha de inicio: 2025-08-25
Fecha alerta: 2025-09-08
Motivo: Paso a segundo plazo de contratación
Jefe (ID): 4678 - RUT: 11.524.484-1
Días en Cramer: 3 días


In [ ]:
# empleado_filtrado = {
#     "person_id": empleado_completo.get("person_id"),
#     "id": empleado_completo.get("id"),
#     "full_name": empleado_completo.get("full_name"),
#     #"document_type": empleado_completo.get("document_type"),
#     #"document_number": empleado_completo.get("document_number"),
#     "rut": empleado_completo.get("rut"),
#     "email": empleado_completo.get("email"),
#     "personal_email": empleado_completo.get("personal_email"),
#     "address": empleado_completo.get("address"),
#     "street": empleado_completo.get("street"),
#     "street_number": empleado_completo.get("street_number"),
#     "city": empleado_completo.get("city"),
#     "district": empleado_completo.get("district"),
#     "region": empleado_completo.get("region"),
#     "phone": empleado_completo.get("phone"),
#     "gender": empleado_completo.get("gender"),
#     "birthday": empleado_completo.get("birthday"),
#     "university": empleado_completo.get("university"),
#     "degree": empleado_completo.get("degree"),
#     "active_since": empleado_completo.get("active_since"),
#     "status": empleado_completo.get("status"),
#     "payment_method": empleado_completo.get("payment_method"),
#     "bank": empleado_completo.get("bank"),
#     "account_type": empleado_completo.get("account_type"),
#     "account_number": empleado_completo.get("account_number"),
#     "nationality": empleado_completo.get("nationality"),
#     "civil_status": empleado_completo.get("civil_status"),
#     "health_company": empleado_completo.get("health_company"),
#     "pension_regime": empleado_completo.get("pension_regime"),
#     "pension_fund": empleado_completo.get("pension_fund"),
#     "afc": empleado_completo.get("afc"),
#     "active_until": empleado_completo.get("active_until"),
#     "termination_reason": empleado_completo.get("termination_reason"),
#         "current_job": {
#             "periodicity": empleado_completo.get("current_job", {}).get("periodicity"),
#             "frequency": empleado_completo.get("current_job", {}).get("frequency"),
#             "working_schedule_type": empleado_completo.get("current_job", {}).get("working_schedule_type"),
#             "previous_job_id": empleado_completo.get("current_job", {}).get("previous_job_id"),
#             "id": empleado_completo.get("current_job", {}).get("id"),
#             "company_id": empleado_completo.get("current_job", {}).get("company_id"),
#             "area_id": empleado_completo.get("current_job", {}).get("area_id"),
#             "contract_type": empleado_completo.get("current_job", {}).get("contract_type"),
#             "start_date": empleado_completo.get("current_job", {}).get("start_date"),
#             "end_date": empleado_completo.get("current_job", {}).get("end_date"),
#             "notice_date": empleado_completo.get("current_job", {}).get("notice_date"),
#             "contract_finishing_date_1": empleado_completo.get("current_job", {}).get("contract_finishing_date_1"),
#             "contract_finishing_date_2": empleado_completo.get("current_job", {}).get("contract_finishing_date_2"),
#         "weekly_hours": empleado_completo.get("current_job", {}).get("weekly_hours"),
#         "cost_center": empleado_completo.get("current_job", {}).get("cost_center"),
#         "active_until": empleado_completo.get("current_job", {}).get("active_until"),
#         "location_id": empleado_completo.get("current_job", {}).get("location_id"),
#         "without_wage": empleado_completo.get("current_job", {}).get("without_wage"),
#         "contract_subscription_date": empleado_completo.get("current_job", {}).get("contract_subscription_date"),
    
#         "contractual_stipulation": empleado_completo.get("current_job", {}).get("contractual_stipulation"),
#         "contractual_detail": empleado_completo.get("current_job", {}).get("contractual_detail"),
#         "currency_code": empleado_completo.get("current_job", {}).get("currency_code"),
#         "reward_concept": empleado_completo.get("current_job", {}).get("reward_concept"),
#         "reward_payment_period": empleado_completo.get("current_job", {}).get("reward_payment_period"),
#         "reward_description": empleado_completo.get("current_job", {}).get("reward_description"),
#         "grado_sector_publico_chile": empleado_completo.get("current_job", {}).get("grado_sector_publico_chile"),
#         "estamento_sector_publico_chile": empleado_completo.get("current_job", {}).get("estamento_sector_publico_chile"),
#         "base_wage": empleado_completo.get("current_job", {}).get("base_wage"),
#         "role": empleado_completo.get("current_job", {}).get("role")

#     }
# }

# data_boss = {
#     "id":empleado_completo.get("current_job", {}).get("boss", {}).get("id"),
#     "rut":empleado_completo.get("current_job", {}).get("boss", {}).get("rut"),
# }